code source:
1. https://www.kaggle.com/code/rodsaldanha/stock-prediction-pytorch/notebook
2. https://www.analyticsvidhya.com/blog/2021/07/understanding-rnn-step-by-step-with-pytorch/

In [65]:
# Importing libraries. 
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import torch
from torch import nn
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import Dataset

## Data

In [2]:
for dirname, _, filenames in os.walk('./archive/stocks'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./archive/stocks/RIV.csv
./archive/stocks/ANTE.csv
./archive/stocks/CSCO.csv
./archive/stocks/PRI.csv
./archive/stocks/NZF.csv
./archive/stocks/HLNE.csv
./archive/stocks/UNT.csv
./archive/stocks/HUBS.csv
./archive/stocks/GPL.csv
./archive/stocks/SBGI.csv
./archive/stocks/UFCS.csv
./archive/stocks/TEAF.csv
./archive/stocks/RJZ.csv
./archive/stocks/GFED.csv
./archive/stocks/AIRT.csv
./archive/stocks/SITC.csv
./archive/stocks/NXN.csv
./archive/stocks/BNY.csv
./archive/stocks/RHE.csv
./archive/stocks/ISSC.csv
./archive/stocks/KNDI.csv
./archive/stocks/EFT.csv
./archive/stocks/TRNS.csv
./archive/stocks/CARO.csv
./archive/stocks/NTZ.csv
./archive/stocks/AEIS.csv
./archive/stocks/UAL.csv
./archive/stocks/TROW.csv
./archive/stocks/ISRG.csv
./archive/stocks/EGP.csv
./archive/stocks/EFC.csv
./archive/stocks/EEX.csv
./archive/stocks/NVR.csv
./archive/stocks/PLYM.csv
./archive/stocks/TELA.csv
./archive/stocks/FEIM.csv
./archive/stocks/WOR.csv
./archive/stocks/TLRY.csv
./archive/stocks/PRGO.csv
./a

./archive/stocks/EV.csv
./archive/stocks/AAP.csv
./archive/stocks/PMBC.csv
./archive/stocks/CXP.csv
./archive/stocks/MVC.csv
./archive/stocks/PTMN.csv
./archive/stocks/MUX.csv
./archive/stocks/ATVI.csv
./archive/stocks/FEI.csv
./archive/stocks/GRIL.csv
./archive/stocks/MRCCL.csv
./archive/stocks/QES.csv
./archive/stocks/MGLN.csv
./archive/stocks/EVRG.csv
./archive/stocks/VBF.csv
./archive/stocks/EA.csv
./archive/stocks/DE.csv
./archive/stocks/MVT.csv
./archive/stocks/CETX.csv
./archive/stocks/FHN.csv
./archive/stocks/SOHO.csv
./archive/stocks/MINI.csv
./archive/stocks/INUV.csv
./archive/stocks/VMI.csv
./archive/stocks/AMS.csv
./archive/stocks/ANH.csv
./archive/stocks/CWH.csv
./archive/stocks/CTS.csv
./archive/stocks/TTI.csv
./archive/stocks/PAYS.csv
./archive/stocks/OFED.csv
./archive/stocks/KOSS.csv
./archive/stocks/SMRT.csv
./archive/stocks/HDB.csv
./archive/stocks/QHC.csv
./archive/stocks/AACG.csv
./archive/stocks/SPG.csv
./archive/stocks/CPAA.csv
./archive/stocks/EMKR.csv
./archive

./archive/stocks/TGP.csv
./archive/stocks/CIA.csv
./archive/stocks/MGR.csv
./archive/stocks/HBANN.csv
./archive/stocks/OCFT.csv
./archive/stocks/SXTC.csv
./archive/stocks/SLF.csv
./archive/stocks/SNY.csv
./archive/stocks/MFV.csv
./archive/stocks/CHE.csv
./archive/stocks/HTGC.csv
./archive/stocks/TG.csv
./archive/stocks/APA.csv
./archive/stocks/BABA.csv
./archive/stocks/ARI.csv
./archive/stocks/VRS.csv
./archive/stocks/RILYZ.csv
./archive/stocks/OCFC.csv
./archive/stocks/DMO.csv
./archive/stocks/SNN.csv
./archive/stocks/MFA.csv
./archive/stocks/WK.csv
./archive/stocks/ASM.csv
./archive/stocks/FCF.csv
./archive/stocks/SXC.csv
./archive/stocks/FFNW.csv
./archive/stocks/XHR.csv
./archive/stocks/EBAYL.csv
./archive/stocks/IBKR.csv
./archive/stocks/BCML.csv
./archive/stocks/UBOH.csv
./archive/stocks/BATL.csv
./archive/stocks/CY.csv
./archive/stocks/K.csv
./archive/stocks/IROQ.csv
./archive/stocks/JWN.csv
./archive/stocks/SXT.csv
./archive/stocks/MESA.csv
./archive/stocks/AFL.csv
./archive/st

In [86]:
filepath = './archive/stocks/AAPL.csv'
data = pd.read_csv(filepath)
data = data.sort_values('Date')
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1980-12-12,0.513393,0.515625,0.513393,0.513393,0.406782,117258400
1,1980-12-15,0.488839,0.488839,0.486607,0.486607,0.385558,43971200
2,1980-12-16,0.453125,0.453125,0.450893,0.450893,0.357260,26432000
3,1980-12-17,0.462054,0.464286,0.462054,0.462054,0.366103,21610400
4,1980-12-18,0.475446,0.477679,0.475446,0.475446,0.376715,18362400


### Normalize data

In [87]:
price = data[['Close']]
price.info()
print(price.shape)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9909 entries, 0 to 9908
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   9909 non-null   float64
dtypes: float64(1)
memory usage: 154.8 KB
(9909, 1)


In [88]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1,1))
price['Close'] = scaler.fit_transform(price['Close'].values.reshape(-1,1))

/var/folders/x3/38g4jf617810gvrj4clgg_mr0000gn/T/ipykernel_7742/3750275899.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price['Close'] = scaler.fit_transform(price['Close'].values.reshape(-1,1))


### Load Data

In [124]:
class Data(Dataset):
    def __init__(self, input_data, seq_len):
        self.input = input_data
        self.seq_len = seq_len
    
    def __getitem__(self,idx):
        return self.input[idx:idx + self.seq_len-1], self.input[idx + self.seq_len-1]
    
    def __len__(self):
        return len(self.input) - self.seq_len

In [122]:
testset_size = int(np.round(0.2*len(price)))
trainset_size = data.shape[0] - (testset_size)

train_input = np.array(price[:trainset_size])
train_input = torch.tensor(train_input, dtype=torch.float)
test_input = np.array(price[trainset_size:])
test_input = torch.tensor(test_input, dtype=torch.float)
# hyperparameters
seq_len = 20
batch_size = 8
trainset = Data(train_input,seq_len)
testset = Data(test_input,seq_len)
train_loader = DataLoader(trainset, batch_size=8)
test_loader = DataLoader(testset, batch_size=8)

In [123]:
for inp, label in train_loader:
    print(inp.numpy().shape)
    print(inp.numpy())
    print(label)
    break

(8, 19, 1)
[[[-0.9980614 ]
  [-0.9982252 ]
  [-0.99844366]
  [-0.9983754 ]
  [-0.99829346]
  [-0.998116  ]
  [-0.9979658 ]
  [-0.9978293 ]
  [-0.9976518 ]
  [-0.99732417]
  [-0.9972696 ]
  [-0.9973651 ]
  [-0.9974744 ]
  [-0.9974334 ]
  [-0.9975153 ]
  [-0.9976791 ]
  [-0.9978293 ]
  [-0.99789757]
  [-0.9977201 ]]

 [[-0.9982252 ]
  [-0.99844366]
  [-0.9983754 ]
  [-0.99829346]
  [-0.998116  ]
  [-0.9979658 ]
  [-0.9978293 ]
  [-0.9976518 ]
  [-0.99732417]
  [-0.9972696 ]
  [-0.9973651 ]
  [-0.9974744 ]
  [-0.9974334 ]
  [-0.9975153 ]
  [-0.9976791 ]
  [-0.9978293 ]
  [-0.99789757]
  [-0.9977201 ]
  [-0.9977474 ]]

 [[-0.99844366]
  [-0.9983754 ]
  [-0.99829346]
  [-0.998116  ]
  [-0.9979658 ]
  [-0.9978293 ]
  [-0.9976518 ]
  [-0.99732417]
  [-0.9972696 ]
  [-0.9973651 ]
  [-0.9974744 ]
  [-0.9974334 ]
  [-0.9975153 ]
  [-0.9976791 ]
  [-0.9978293 ]
  [-0.99789757]
  [-0.9977201 ]
  [-0.9977474 ]
  [-0.99787027]]

 [[-0.9983754 ]
  [-0.99829346]
  [-0.998116  ]
  [-0.9979658 ]
  [-0.9